In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow import keras 
import matplotlib.pyplot as plt

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("data/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [8]:
application_df.isnull().sum()

EIN                       0
NAME                      0
APPLICATION_TYPE          0
AFFILIATION               0
CLASSIFICATION            0
USE_CASE                  0
ORGANIZATION              0
STATUS                    0
INCOME_AMT                0
SPECIAL_CONSIDERATIONS    0
ASK_AMT                   0
IS_SUCCESSFUL             0
dtype: int64

In [9]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [11]:
# Choose a cutoff value and create a list of application types to be replaced
# cutoff = 528
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [12]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [14]:
# CLASSIFICATION binning - create a list of classifications to be replaced

classifications_to_replace = ['C2700', 'C7000', 'C7200',
       'C1700', 'C4000', 'C7100', 'C2800', 'C6000', 'C1238',
       'C5000', 'C7120', 'C1800', 'C4100', 'C1400', 'C1270', 'C2300',
       'C8200', 'C1500', 'C7210', 'C1300', 'C1230', 'C1280', 'C1240',
       'C2710', 'C2561', 'C1250', 'C8000', 'C1245', 'C1260', 'C1235',
       'C1720', 'C1257', 'C4500', 'C2400', 'C8210', 'C1600', 'C1278',
       'C1237', 'C4120', 'C2170', 'C1728', 'C1732', 'C2380', 'C1283',
       'C1570', 'C2500', 'C1267', 'C3700', 'C1580', 'C2570', 'C1256',
       'C1236', 'C1234', 'C1246', 'C2190', 'C4200', 'C0', 'C3200',
       'C5200', 'C1370', 'C2600', 'C1248', 'C6100', 'C1820', 'C1900',
       'C2150']

In [15]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [16]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE','AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'], prefix='DMMY')

In [19]:
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,DMMY_Other,DMMY_T10,DMMY_T19,DMMY_T3,DMMY_T4,DMMY_T5,DMMY_T6,...,DMMY_1-9999,DMMY_10000-24999,DMMY_100000-499999,DMMY_10M-50M,DMMY_1M-5M,DMMY_25000-99999,DMMY_50M+,DMMY_5M-10M,DMMY_N,DMMY_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [17]:
application_df.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'DMMY_Other', 'DMMY_T10',
       'DMMY_T19', 'DMMY_T3', 'DMMY_T4', 'DMMY_T5', 'DMMY_T6', 'DMMY_T7',
       'DMMY_T8', 'DMMY_CompanySponsored', 'DMMY_Family/Parent',
       'DMMY_Independent', 'DMMY_National', 'DMMY_Other', 'DMMY_Regional',
       'DMMY_C1000', 'DMMY_C1200', 'DMMY_C2000', 'DMMY_C2100', 'DMMY_C3000',
       'DMMY_Other', 'DMMY_CommunityServ', 'DMMY_Heathcare', 'DMMY_Other',
       'DMMY_Preservation', 'DMMY_ProductDev', 'DMMY_Association',
       'DMMY_Co-operative', 'DMMY_Corporation', 'DMMY_Trust', 'DMMY_0',
       'DMMY_1-9999', 'DMMY_10000-24999', 'DMMY_100000-499999', 'DMMY_10M-50M',
       'DMMY_1M-5M', 'DMMY_25000-99999', 'DMMY_50M+', 'DMMY_5M-10M', 'DMMY_N',
       'DMMY_Y'],
      dtype='object')

In [18]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
y = application_df['IS_SUCCESSFUL']
X = application_df[['STATUS', 'ASK_AMT', 'DMMY_Other', 'DMMY_T10',
       'DMMY_T19', 'DMMY_T3', 'DMMY_T4', 'DMMY_T5', 'DMMY_T6', 'DMMY_T7',
       'DMMY_T8', 'DMMY_CompanySponsored', 'DMMY_Family/Parent',
       'DMMY_Independent', 'DMMY_National', 'DMMY_Other', 'DMMY_Regional',
       'DMMY_C1000', 'DMMY_C1200', 'DMMY_C2000', 'DMMY_C2100', 'DMMY_C3000',
       'DMMY_Other', 'DMMY_CommunityServ', 'DMMY_Heathcare', 'DMMY_Other',
       'DMMY_Preservation', 'DMMY_ProductDev', 'DMMY_Association',
       'DMMY_Co-operative', 'DMMY_Corporation', 'DMMY_Trust', 'DMMY_0',
       'DMMY_1-9999', 'DMMY_10000-24999', 'DMMY_100000-499999', 'DMMY_10M-50M',
       'DMMY_1M-5M', 'DMMY_25000-99999', 'DMMY_50M+', 'DMMY_5M-10M', 'DMMY_N',
       'DMMY_Y']]
# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [20]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [21]:
X_train.shape

(25724, 55)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=55))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn.summary()